In [1]:
import pandas as pd
df=pd.read_csv("../data/processed/final_merged_file.csv")

In [2]:
df['match_date'] = pd.to_datetime(df['match_date'])

In [3]:
home_team=df.groupby(["home_team_api_id","season","match_date"]).agg(match_count=("id","count")).reset_index().rename(columns={"home_team_api_id":"team_id"})
away_team=df.groupby(["away_team_api_id","season","match_date"]).agg(match_count=("id","count")).reset_index().rename(columns={"away_team_api_id":"team_id"})

In [4]:
z=pd.concat([home_team,away_team]).groupby(["team_id","season","match_date"]).agg({"match_count":"sum"}).reset_index()

In [5]:
z["rest_days"]=z.groupby(["team_id","season"])["match_date"].diff().dt.days

In [6]:
z=z.sort_values(by=["match_date","team_id","season"])

In [7]:
def function(x):
    if pd.isna(x):
        return None
    elif x <=3:
        return "High congestion"
    elif 4<=x<=6:
        return "Medium congestion"
    else:
        return "Normal rest"
z["workload_categories"]=z["rest_days"].apply(function)

In [8]:
#Average rest days per team per season
z.groupby(["season","team_id"]).agg(avg_rest_day=("rest_days","mean")).reset_index()

,season,team_id,avg_rest_day
0,2008/2009,4049,8.272727
1,2008/2009,7819,7.945946
2,2008/2009,7947,8.272727
3,2008/2009,8177,8.454545
4,2008/2009,8178,8.484848
...,...,...,...
636,2015/2016,10252,7.594595
637,2015/2016,10260,7.648649
638,2015/2016,10261,7.567568
639,2015/2016,10269,8.242424


In [9]:
#Number of high-congestion matches per team
z[z["workload_categories"]=="High congestion"].groupby("team_id").agg({"workload_categories":"count"}).reset_index()

,team_id,workload_categories
0,1773,3
1,4087,13
2,4170,3
3,6351,2
4,6391,4
...,...,...
139,10252,32
140,10260,38
141,10261,23
142,10269,18


In [10]:
#Congestion Summary by Season
z.groupby(["workload_categories","season"]).size().reset_index()

,workload_categories,season,0
0,High congestion,2008/2009,239
1,High congestion,2009/2010,286
2,High congestion,2010/2011,184
3,High congestion,2011/2012,187
4,High congestion,2012/2013,180
5,High congestion,2013/2014,161
6,High congestion,2014/2015,208
7,High congestion,2015/2016,206
8,Medium congestion,2008/2009,706
9,Medium congestion,2009/2010,731


In [11]:
#To quantify how performance changes under different workload (congestion) levels.
df.groupby("home_team_api_id").agg(home_goals=("home_team_goal","sum")).reset_index()
df.groupby("away_team_api_id").agg(away_goals=("away_team_goal","sum")).reset_index()

,away_team_api_id,away_goals
0,1773,903
1,4049,188
2,4087,1281
3,4170,251
4,6351,198
...,...,...
144,10260,4007
145,10261,2140
146,10269,3460
147,108893,68


In [12]:
home_z=df.rename(columns={"home_team_api_id":"team_id","home_team_goal":"goals_per","away_team_goal":"goals_against"})[["team_id","season","match_date","goals_per","goals_against"]]
away_z=df.rename(columns={"away_team_api_id":"team_id","away_team_goal":"goals_per","home_team_goal":"goals_against"})[["team_id","season","match_date","goals_per","goals_against"]]

In [13]:
total=pd.concat([home_z,away_z],ignore_index=True).groupby(["team_id","season","match_date"]).agg({"goals_per":"sum","goals_against":"sum"}).reset_index()

In [14]:
total=total.sort_values(by=["team_id","season","match_date"],ascending=True)

In [15]:
z.dtypes

team_id                         int64
season                         object
match_date             datetime64[ns]
match_count                     int64
rest_days                     float64
workload_categories            object
dtype: object

In [16]:
total.dtypes

team_id                   int64
season                   object
match_date       datetime64[ns]
goals_per                 int64
goals_against             int64
dtype: object

In [17]:
t=pd.merge(total,z,on=["team_id","season","match_date"],how="left")

In [18]:
t.isna().sum()

team_id                  0
season                   0
match_date               0
goals_per                0
goals_against            0
match_count              0
rest_days              641
workload_categories    641
dtype: int64

In [19]:
t=t.dropna(subset="rest_days")

In [20]:
t["goal_diff"]=t["goals_per"]-t["goals_against"]

In [21]:
#Performance by Workload Category (Overall)
t.groupby("workload_categories").agg({"goals_per":"mean","goals_against":"mean","goal_diff":"mean"}).reset_index().sort_values(by=["goals_per","goals_against","goal_diff"],ascending=False)

,workload_categories,goals_per,goals_against,goal_diff
0,High congestion,23.293156,20.834040,2.459116
2,Normal rest,22.374057,22.548060,-0.174003
1,Medium congestion,21.667902,21.966234,-0.298332


In [22]:
#Team Resilience Snapshot
t.groupby(["workload_categories","team_id"]).agg({"goal_diff":"count"}).reset_index()

,workload_categories,team_id,goal_diff
0,High congestion,1773,3
1,High congestion,4087,13
2,High congestion,4170,3
3,High congestion,6351,2
4,High congestion,6391,4
...,...,...,...
436,Normal rest,10260,181
437,Normal rest,10261,166
438,Normal rest,10269,186
439,Normal rest,108893,29
